In [111]:
import pandas as pd, numpy as np
import geopandas as gpd, topojson as tp
import matplotlib.pyplot as plt, seaborn as sns

data = pd.read_csv(
    "./sa4_census.csv",
    index_col="code"
)
data.loc[:, "growth"] = (data.loc[:, "2021"] / data.loc[:, "2020"] - 1) * 100
data = data[["name", "2021", "growth"]]
data.columns = ["area", "population", "growth"]
data = data.loc[
    [
        code for code in data.index if (
            (code < 300) |
            (code == 801)
        )
    ],
    :
]

sa4s = gpd.read_file(
    "SA4_2021_AUST_SHP_GDA2020.zip"
).iloc[:, [0, 1, 12]]
sa4s.columns = ["code", "name", "geometry"]
sa4s = sa4s[sa4s["code"].str.isnumeric()]
sa4s["code"] = sa4s["code"].astype("int")
sa4s.set_index("code", inplace=True)
sa4s = sa4s.loc[
    [
        code for code in sa4s.index if (
            (code < 300) |
            (code == 801)
        )
    ],
    :
]

sa4s.loc[:, "population"] = data.loc[:, "population"].copy()
sa4s.loc[:, "growth"] = data.loc[:, "growth"].copy()
sa4s = sa4s[["name", "population", "growth", "geometry"]]
sa4s.dropna(inplace=True)
sa4s["name"] = sa4s["name"].apply(lambda name: name.replace(" - ", " — ").replace(" and ", " & "))
sa4s[["name", "population", "growth"]].to_csv("sa4s.csv")

In [112]:
topo = tp.Topology(
    sa4s.reset_index(),
    toposimplify=1e-3
).topoquantize(1e5)
topo.to_json("sa4s.topojson")